In [1]:
import os
from tensorflow.python.client import device_lib
import nltk
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import keras
from keras.utils import np_utils
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax, BatchNormalization
from keras.callbacks import CSVLogger, ModelCheckpoint
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

2022-11-21 09:32:14.129358: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
os.environ['CUDA_VISIBLE_DEVICES']='0'
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9308486579562545529
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 19434962944
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9430742986068178013
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3090 Ti, pci bus id: 0000:02:00.0, compute capability: 8.6"
]


2022-11-21 09:32:26.391472: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-21 09:32:26.400846: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-11-21 09:32:26.421917: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: NVIDIA GeForce RTX 3090 Ti computeCapability: 8.6
coreClock: 1.905GHz coreCount: 84 deviceMemorySize: 23.69GiB deviceMemoryBandwidth: 938.86GiB/s
2022-11-21 09:32:26.422250: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-11-21 09:32:26.434956: I tensorflow/stream_executor/platform/default/dso_

In [3]:
## Read Data into pd dataframe
import pandas as pd
import json

emotion_df = pd.read_csv('emotion.csv')
di_df = pd.read_csv('data_identification.csv')
sample_sub_df = pd.read_csv('sampleSubmission.csv')
tweets_df = pd.read_pickle("tweets_df.pkl")

In [4]:
t_data = pd.merge(tweets_df, emotion_df)
t_data

,hashtags,tweet_id,text,emotion
0,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...",anticipation
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",sadness
2,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,fear
3,"[authentic, LaughOutLoud]",0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,joy
4,[],0x2c91a8,Still waiting on those supplies Liscus. <LH>,anticipation
...,...,...,...,...
1455558,"[NoWonder, Happy]",0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,joy
1455559,[],0x38959e,In every circumtance I'd like to be thankful t...,joy
1455560,[blessyou],0x2cbca6,there's currently two girls walking around the...,joy
1455561,[],0x24faed,"Ah, corporate life, where you can date <LH> us...",joy


In [5]:
t_data_sample = t_data.sample(n=100)
t_data_sample

,hashtags,tweet_id,text,emotion
41563,[RanbirKapoor],0x2db88f,T834: I don't know why bad things are more hap...,joy
1374274,"[GameOfThones7, Insecure]",0x2912a4,So I watched yesterday's episode of #GameOfTho...,fear
1145835,[],0x34f07c,What is it with dokkan on Apple I get better p...,disgust
440919,"[XtraXtraLove, MercyChinwo]",0x376008,"Excess love. Your love is kind, Your love is p...",trust
1420229,"[prochoice, Safe, OnDemandNoApologies, Governm...",0x232989,As fast as #prochoice went from #Safe&Rare to ...,trust
...,...,...,...,...
489632,[WOKENWisdom],0x2cf73c,@MATTHARDYBRAND Can your #WOKENWisdom explain ...,sadness
912367,[],0x1d4175,PrankInvasion CALLS OUT h3h3Productions | <LH...,surprise
1181239,[],0x1f8e7d,@pamera_chen <LH> WIFEY💗,joy
602382,[],0x352a65,We are creatures of habit. Praising <LH> is ha...,anticipation


In [5]:
import spacy
import spacy.cli

spacy.cli.download("en_core_web_md")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 MB 10.2 MB/s eta 0:00:00
  Attempting uninstall: en-core-web-md
    Found existing installation: en-core-web-md 3.3.0
    Uninstalling en-core-web-md-3.3.0:
      Successfully uninstalled en-core-web-md-3.3.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [6]:
nlp = spacy.load('en_core_web_md', disable=["ner", "parser"])

In [7]:
for token in  nlp("Apple is looking at buying U.K. startup for $1 billion"):
  print('Word: {}  --> {}'.format(token.text, token.lemma_))

Word: Apple  --> Apple
Word: is  --> be
Word: looking  --> look
Word: at  --> at
Word: buying  --> buy
Word: U.K.  --> U.K.
Word: startup  --> startup
Word: for  --> for
Word: $  --> $
Word: 1  --> 1
Word: billion  --> billion


In [8]:
from gensim.models import Word2Vec
model = Word2Vec.load("wiki-lemma-100D-phrase")

In [11]:
model.wv.most_similar('Google', topn = 5)

[('Microsoft', 0.8559668660163879),
 ('Google_Search', 0.8424477577209473),
 ('Google_Analytics', 0.8412379026412964),
 ('Yahoo', 0.8326561450958252),
 ('AltaVista', 0.824895977973938)]

In [10]:
# import numpy as np
# # Get all words from wiki word2vec model
# model_words = set(model.wv.index_to_key)

# def text2vec_raw(w2vmodel, text):
#     doc = nlp(text)
#     # convert a movie review into vectors
#     text_vecs = [w2vmodel.wv[word.lemma_] for word in doc if word.lemma_ in model_words]
#     # calculate the mean of the vectors and return

#     if len(text_vecs) > 1:
#         res =  np.mean(text_vecs, axis = 0)
#         return res
#     elif len(text_vecs) == 1:
#         return text_vecs[0]
#     else:
#         return np.nan

In [25]:
model.wv.save_word2vec_format('keras_word2vec.txt', binary=False)

In [6]:
import numpy as np
# load the whole embedding into memory
embeddings_index = dict()
with open('keras_word2vec.txt') as f:
  for line in f:
    values = line.split()
    # 只接受長度為 101 的向量 (word + 100d embedding)
    if len(values) != 101:
      print(values)
      continue
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

['346848', '100']


In [22]:
from tqdm import tqdm
import spacy.cli
spacy.cli.download("en_core_web_md")
nlp = spacy.load('en_core_web_md', disable=["ner", "parser"])

review_lines = list()
reviews = t_data_sample['text'].values.tolist()

for review in tqdm(reviews):   
    doc = nlp(review)
    words = [word.lemma_ for word in doc]
    review_lines.append(words)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 MB 4.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 236.52it/s]


In [23]:
np.save("test_review_lines", review_lines)

/home/eric/anaconda3/envs/dm_tf_2.5_py_3.7_spacy/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [24]:
test_review_lines = np.load("test_review_lines.npy", allow_pickle=True).tolist()

In [25]:
review_lines

[['T834',
  ':',
  'I',
  'do',
  'not',
  'know',
  'why',
  'bad',
  'thing',
  'be',
  'more',
  'happy',
  'than',
  'good',
  'thing',
  '.',
  'to',
  'be',
  'happy',
  'we',
  'need',
  'to',
  'be',
  'bad',
  'or',
  'what',
  '?',
  '<',
  'LH',
  '>',
  '#',
  'RanbirKapoor',
  '<',
  'LH',
  '>'],
 ['so',
  'I',
  'watch',
  'yesterday',
  "'s",
  'episode',
  'of',
  '#',
  'gameofthones7',
  'and',
  'now',
  'I',
  'be',
  'going',
  'to',
  'watch',
  '#',
  'insecure',
  '.',
  'be',
  'back',
  'later',
  'to',
  'share',
  'my',
  'thought'],
 ['what',
  'be',
  'it',
  'with',
  'dokkan',
  'on',
  'Apple',
  'I',
  'get',
  'well',
  'pull',
  'then',
  'android',
  '...',
  '<',
  'LH',
  '>'],
 ['excess',
  'love',
  '.',
  'your',
  'love',
  'be',
  'kind',
  ',',
  'your',
  'love',
  'be',
  'patient',
  '.',
  'Jesus',
  'you',
  'love',
  'I',
  'too',
  '/',
  'excessively',
  'much',
  'oooo',
  '......',
  '#inspire',
  '#',
  'xtraxtralove',
  '#',
  '

In [26]:
test_review_lines

[['T834',
  ':',
  'I',
  'do',
  'not',
  'know',
  'why',
  'bad',
  'thing',
  'be',
  'more',
  'happy',
  'than',
  'good',
  'thing',
  '.',
  'to',
  'be',
  'happy',
  'we',
  'need',
  'to',
  'be',
  'bad',
  'or',
  'what',
  '?',
  '<',
  'LH',
  '>',
  '#',
  'RanbirKapoor',
  '<',
  'LH',
  '>'],
 ['so',
  'I',
  'watch',
  'yesterday',
  "'s",
  'episode',
  'of',
  '#',
  'gameofthones7',
  'and',
  'now',
  'I',
  'be',
  'going',
  'to',
  'watch',
  '#',
  'insecure',
  '.',
  'be',
  'back',
  'later',
  'to',
  'share',
  'my',
  'thought'],
 ['what',
  'be',
  'it',
  'with',
  'dokkan',
  'on',
  'Apple',
  'I',
  'get',
  'well',
  'pull',
  'then',
  'android',
  '...',
  '<',
  'LH',
  '>'],
 ['excess',
  'love',
  '.',
  'your',
  'love',
  'be',
  'kind',
  ',',
  'your',
  'love',
  'be',
  'patient',
  '.',
  'Jesus',
  'you',
  'love',
  'I',
  'too',
  '/',
  'excessively',
  'much',
  'oooo',
  '......',
  '#inspire',
  '#',
  'xtraxtralove',
  '#',
  '

In [57]:
sentiment = t_data_sample['emotion'].values
sentiment

array(['anticipation', 'sadness', 'disgust', ..., 'joy', 'anticipation',
       'joy'], dtype=object)

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(review_lines, t_data_sample['emotion'],
                                   random_state=104, 
                                   test_size=0.25, 
                                   shuffle=True)

In [50]:
print("x_train.shape: ", len(X_train))
print("y_train.shape: ", y_train.shape)
print("x_test.shape: ", len(X_test))
print("y_test.shape: ", y_test.shape)

x_train.shape:  75000
y_train.shape:  (75000,)
x_test.shape:  25000
y_test.shape:  (25000,)


In [59]:
## deal with label (string -> one-hot)

from sklearn.preprocessing import LabelEncoder
import keras
from keras.utils import np_utils

label_encoder = LabelEncoder()
label_encoder.fit(sentiment)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:4]:\n', sentiment[0:4])
print('\ny_train.shape: ', sentiment.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    return np_utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

sentiment_oh = label_encode(label_encoder, sentiment)

print('\n\n## After convert')
print('y_train[0:4]:\n', sentiment_oh[0:4])
print('\ny_train.shape: ', sentiment_oh.shape)


check label:  ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']

## Before convert
y_train[0:4]:
 ['anticipation' 'sadness' 'disgust' 'sadness']

y_train.shape:  (100000,)


## After convert
y_train[0:4]:
 [[0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]]

y_train.shape:  (100000, 8)


In [64]:
import numpy as np
### 找出影評共有幾個 words
sentence_lengths = t_data_sample['text'].apply(lambda x: len(x.split()))
quantiles = np.percentile(sentence_lengths, [25, 50, 75])

print('The minimal word count in a movie review: ', min(sentence_lengths))
print('The first quantile (25%) of word count in a movie review:', quantiles[0])
print('The second quantile (50%) of word count in a movie review:', quantiles[1])
print('The third quantile (75%) of word count in a movie review:', quantiles[2])
print('The maximum word count in a movie review: ', max(sentence_lengths))

The minimal word count in a movie review:  1
The first quantile (25%) of word count in a movie review: 10.0
The second quantile (50%) of word count in a movie review: 15.0
The third quantile (75%) of word count in a movie review: 20.0
The maximum word count in a movie review:  43


In [66]:
import numpy as np
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

# 初始化 tokenizer
# 將文字轉換成 2D integer tensor
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(review_lines)
sequences = tokenizer_obj.texts_to_sequences(review_lines)

# 因為model的input必須一樣長，所以要使用padding，句子短於512的都會補0，長於512的會被truncate
word_index = tokenizer_obj.word_index
print('Found %s unique tokens.' % len(word_index))
review_pad = pad_sequences(sequences, maxlen=50)

print('Shape of review tensor:', review_pad.shape)
print('Shape of sentiment tensor:', sentiment_oh.shape)


Found 119720 unique tokens.
Shape of review tensor: (100000, 50)
Shape of sentiment tensor: (100000, 8)


In [67]:
vocab_size = len(word_index) + 1
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer_obj.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [69]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Embedding
from keras.initializers import Constant
import tensorflow as tf


# define model
gru_model = Sequential()
e = Embedding(vocab_size, 100, embeddings_initializer=Constant(embedding_matrix), trainable=False)
gru_model.add(e)
gru_model.add(GRU(64, dropout = 0.2, recurrent_dropout = 0.2))
gru_model.add(Dense(8, activation='softmax'))
# compile the model
gru_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# summarize the model
print(gru_model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         11972100  
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                31872     
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 520       
Total params: 12,004,492
Trainable params: 32,392
Non-trainable params: 11,972,100
_________________________________________________________________
None


In [70]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
csv_logger = CSVLogger('logs/training_log_lstm.csv')
callbacks = [ModelCheckpoint(filepath='./model_lstm/model_lstm_{epoch}.h5', verbose=1, save_best_only=True), csv_logger, tensorboard_callback]

# fit the model
gru_model.fit(
    review_pad, sentiment_oh, 
    batch_size=128, 
    epochs=10, 
    validation_split=0.1, 
    verbose=1, 
    callbacks=callbacks)
# evaluate the model

2022-11-21 00:06:08.728056: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-11-21 00:06:08.728115: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-11-21 00:06:08.728248: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1611] Profiler found 1 GPUs
2022-11-21 00:06:08.731856: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory
2022-11-21 00:06:08.732902: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so'; dlerror: libcupti.so: cannot open shared object file: No such file or directory
2022-11-21 00:06:08.732973: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1661] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be 

Epoch 1/10


2022-11-21 00:06:16.478165: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11


  1/704 [..............................] - ETA: 41:03 - loss: 2.1417 - accuracy: 0.0938

2022-11-21 00:06:17.654116: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2022-11-21 00:06:17.654205: I tensorflow/stream_executor/cuda/cuda_blas.cc:1838] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2022-11-21 00:06:17.970079: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-11-21 00:06:17.970128: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-11-21 00:06:17.970187: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1661] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.


  2/704 [..............................] - ETA: 4:32 - loss: 2.1201 - accuracy: 0.1211 

2022-11-21 00:06:18.228581: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-11-21 00:06:18.229197: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1752] function cupti_interface_->Finalize()failed with error CUPTI could not be loaded or symbol could not be found.
2022-11-21 00:06:18.402701: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-11-21 00:06:18.471690: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2022-11-21 00:06:18.511932: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/train/plugins/profile/2022_11_21_00_06_18
2022-11-21 00:06:18.521183: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_11_21_00_06_18/nasic02.trace.json.gz
2022-11-21 00:06:18.677887: I tensorflow/core/profi

704/704 [==============================] - 130s 179ms/step - loss: 1.6421 - accuracy: 0.3952 - val_loss: 1.5314 - val_accuracy: 0.4410

Epoch 00001: val_loss improved from inf to 1.53144, saving model to ./model_lstm/model_lstm_1.h5
Epoch 2/10
704/704 [==============================] - 127s 180ms/step - loss: 1.5276 - accuracy: 0.4398 - val_loss: 1.4949 - val_accuracy: 0.4560

Epoch 00002: val_loss improved from 1.53144 to 1.49492, saving model to ./model_lstm/model_lstm_2.h5
Epoch 3/10
704/704 [==============================] - 121s 171ms/step - loss: 1.4927 - accuracy: 0.4530 - val_loss: 1.4623 - val_accuracy: 0.4695

Epoch 00003: val_loss improved from 1.49492 to 1.46228, saving model to ./model_lstm/model_lstm_3.h5
Epoch 4/10
704/704 [==============================] - 125s 178ms/step - loss: 1.4677 - accuracy: 0.4644 - val_loss: 1.4419 - val_accuracy: 0.4746

Epoch 00004: val_loss improved from 1.46228 to 1.44192, saving model to ./model_lstm/model_lstm_4.h5
Epoch 5/10
704/704 [===

In [71]:
%load_ext tensorboard

In [74]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 2027), started 0:01:43 ago. (Use '!kill 2027' to kill it.)

In [75]:
model_loads = keras.models.load_model('model_lstm/model_lstm_10.h5')

In [86]:
review_pad[0].shape

(50,)

In [126]:
pc = gru_model.predict(review_pad[0:500])

In [129]:
pred_result = label_decode(label_encoder, pc)

In [113]:
pc_argmax = np.argmax(pc, axis=0)
pc_argmax.shape

(8,)

In [130]:
from sklearn.metrics import accuracy_score

print('testing accuracy: {}'.format(round(accuracy_score(label_decode(label_encoder, sentiment_oh[0:500]), pred_result), 2)))

testing accuracy: 0.5
